# Bike Isochrone

In [ ]:
import json
import time
import os
import geopandas as gpd
from openrouteservice import Client
from openrouteservice.exceptions import ApiError

def calculate_isochrones_bike(input_file, output_path, output_filename, print_statement):
    # Load poi data with GeoPandas
    poi = gpd.read_file(input_file)

    # Calculate centroids for each polygon
    poi['centroid'] = poi['geometry'].centroid

    # Change CRS to EPSG:4326
    poi = poi.to_crs("EPSG:4326")

    # Get the name of the ID column
    poi_id_column = 'id'  # Assuming the column name is 'id'

    # Initialize OpenRouteService client
    api_key = "5b3ce3597851110001cf6248ec8fc0286b4a4fd1a90fc8108327d823"  # Replace with your actual API key
    client = Client(key=api_key)

    # Define parameters for isochrone calculation
    profile = 'cycling-regular'
    range_type = 'time'
    range_seconds = [900, 1800]  # 15 and 30 Minutes

    # Create the directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # Function to calculate isochrones with retry mechanism
    def calculate_isochrones_with_retry(coordinate):
        retries = 3  # Number of retries
        for attempt in range(retries):
            try:
                isochrones = client.isochrones(locations=[coordinate], profile=profile, range_type=range_type, range=range_seconds, attributes=['total_pop'])
                return isochrones
            except ApiError as e:
                if "rate limit exceeded" in str(e):
                    print(f"Rate limit exceeded. Retrying in 10 seconds...")
                    time.sleep(10)  # Retry after 10 seconds for rate limit errors
                elif "Unable to build an isochrone map" in str(e):
                    print(f"Unable to build an isochrone map for POI. Skipping...")
                    return None
                else:
                    raise e  # Re-raise the exception if it's not related to rate limit exceeded
        return None

    # Iterate over poi and calculate isochrones
    for idx, row in poi.iterrows():
        # Get centroid coordinates of the poi
        lon, lat = row['centroid'].x, row['centroid'].y
        coordinate = (lon, lat)

        # Calculate isochrones with retry mechanism
        isochrones = calculate_isochrones_with_retry(coordinate)
        if isochrones is None:
            continue  # Skip the poi if isochrones cannot be calculated
            
        # Get the poi ID from the column
        poi_id = row[poi_id_column]

        # Add poi ID to each isochrone feature
        for feature in isochrones['features']:
            feature['properties']['poi_id'] = poi_id

        # Save isochrones data to file
        output_file_path = os.path.join(output_path, f'{output_filename}_{idx + 1}.geojson')
        
        with open(output_file_path, 'w') as output_file:
            json.dump(isochrones, output_file)

        print(print_statement.format(idx))

        # Rate limit handling
        time.sleep(3)  # Add a small delay to avoid rate limits and respect the rate limit

    print("Isochrone calculation completed.")


## Green spaces
### Barcelona

definitive wheelchair access

In [ ]:
input_file = '../data/output/barcelona/access/wheelchair_greenspace.geojson'
output_path = '../data/output/barcelona/isochrones_bike_greenspaces_wheelchair/'
output_filename = 'b_iso_bike_greenspaces_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_bike(input_file, output_path, output_filename, print_statement)

with ramps

In [ ]:
input_file = '../data/output/barcelona/access/combined_greenspaces_wheelchair.geojson'
output_path = '../data/output/barcelona/isochrones_bike_greenspaces/'
output_filename = 'b_iso_bike_greenspaces'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_bike(input_file, output_path, output_filename, print_statement)

### Munich

In [ ]:
input_file = '../data/output/munich/access/wheelchair_greenspace.geojson'
output_path = '../data/output/munich/isochrones_bike_greenspaces_wheelchair/'
output_filename = 'b_iso_bike_greenspaces_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_bike(input_file, output_path, output_filename, print_statement)

input_file = '../data/output/munich/access/combined_greenspaces_wheelchair.geojson'
output_path = '../data/output/munich/isochrones_bike_greenspaces/'
output_filename = 'b_iso_bike_greenspaces'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_bike(input_file, output_path, output_filename, print_statement)

## Public spaces
### Barcelona

definitive wheelchair access

In [ ]:
input_file = '../data/output/barcelona/access/wheelchair_publicspaces.geojson'
output_path = '../data/output/barcelona/isochrones_bike_public_spaces_wheelchair/'
output_filename = 'b_iso_bike_public_spaces_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_bike(input_file, output_path, output_filename, print_statement)

with nulls

In [ ]:
input_file = '../data/output/barcelona/access/combined_public_spaces_wheelchair.geojson'
output_path = '../data/output/barcelona/isochrones_bike_public_spaces/'
output_filename = 'b_iso_bike_public_spaces'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_bike(input_file, output_path, output_filename, print_statement)

### Munich

In [ ]:
input_file = '../data/output/munich/access/wheelchair_publicspaces.geojson'
output_path = '../data/output/munich/isochrones_bike_public_spaces_wheelchair/'
output_filename = 'b_iso_bike_public_spaces_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_bike(input_file, output_path, output_filename, print_statement)

In [ ]:
input_file = '../data/output/munich/access/combined_public_spaces_wheelchair.geojson'
output_path = '../data/output/munich/isochrones_bike_public_spaces/'
output_filename = 'b_iso_bike_public_spaces'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_bike(input_file, output_path, output_filename, print_statement)

## Public building
### Barcelona

definitive wheelchair access

In [ ]:
input_file = '../data/output/barcelona/access/wheelchair_public_building.geojson'
output_path = '../data/output/barcelona/isochrones_bike_public_building_wheelchair/'
output_filename = 'b_iso_bike_public_building_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_bike(input_file, output_path, output_filename, print_statement)

with nulls

In [ ]:
input_file = '../data/output/barcelona/access/combined_public_building_wheelchair.geojson'
output_path = '../data/output/barcelona/isochrones_bike_public_building/'
output_filename = 'b_iso_bike_public_building'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_bike(input_file, output_path, output_filename, print_statement)

### Munich

In [ ]:
input_file = '../data/output/munich/access/wheelchair_public_building.geojson'
output_path = '../data/output/munich/isochrones_bike_public_building_wheelchair/'
output_filename = 'b_iso_bike_public_building_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_bike(input_file, output_path, output_filename, print_statement)

In [ ]:
input_file = '../data/output/munich/access/combined_public_building_wheelchair.geojson'
output_path = '../data/output/munich/isochrones_bike_public_building/'
output_filename = 'b_iso_bike_public_building'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_bike(input_file, output_path, output_filename, print_statement)

# Walking Isochrones

In [ ]:
import json
import time
import os
import geopandas as gpd
from openrouteservice import Client
from openrouteservice.exceptions import ApiError

def calculate_isochrones_walk(input_file, output_path, output_filename, print_statement):
    # Load poi data with GeoPandas
    poi = gpd.read_file(input_file)

    # Calculate centroids for each polygon
    poi['centroid'] = poi['geometry'].centroid

    # Change CRS to EPSG:4326
    poi = poi.to_crs("EPSG:4326")
    
    # Get the name of the ID column
    poi_id_column = 'id'  # Assuming the column name is 'id'

    # Initialize OpenRouteService client
    api_key = "5b3ce3597851110001cf6248ec8fc0286b4a4fd1a90fc8108327d823"  # Replace with your actual API key
    client = Client(key=api_key)

    # Define parameters for isochrone calculation
    profile = 'foot-walking'
    range_type = 'time'
    range_seconds = [900, 1800]  # 15 and 30 Minutes

    # Create the directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # Function to calculate isochrones with retry mechanism
    def calculate_isochrones_with_retry(coordinate):
        retries = 3  # Number of retries
        for attempt in range(retries):
            try:
                isochrones = client.isochrones(locations=[coordinate], profile=profile, range_type=range_type, range=range_seconds, attributes=['total_pop'])
                return isochrones
            except ApiError as e:
                if "rate limit exceeded" in str(e):
                    print(f"Rate limit exceeded. Retrying in 10 seconds...")
                    time.sleep(10)  # Retry after 10 seconds for rate limit errors
                elif "Unable to build an isochrone map" in str(e):
                    print(f"Unable to build an isochrone map for POI. Skipping...")
                    return None
                else:
                    raise e  # Re-raise the exception if it's not related to rate limit exceeded
        return None

    # Iterate over poi and calculate isochrones
    for idx, row in poi.iterrows():
        # Get centroid coordinates of the poi
        lon, lat = row['centroid'].x, row['centroid'].y
        coordinate = (lon, lat)

        # Calculate isochrones with retry mechanism
        isochrones = calculate_isochrones_with_retry(coordinate)
        if isochrones is None:
            continue  # Skip the poi if isochrones cannot be calculated
            
        # Get the poi ID from the column
        poi_id = row[poi_id_column]            

        # Add poi ID to each isochrone feature
        for feature in isochrones['features']:
            feature['properties']['poi_id'] = poi_id

        # Save isochrones data to file
        output_file_path = os.path.join(output_path, f'{output_filename}_{idx + 1}.geojson')
        
        with open(output_file_path, 'w') as output_file:
            json.dump(isochrones, output_file)

        print(print_statement.format(idx))

        # Rate limit handling
        time.sleep(3)  # Add a small delay to avoid rate limits and respect the rate limit

    print("Isochrone calculation completed.")


## Green spaces
### Barcelona

definitive wheelchair

In [ ]:
input_file = '../data/output/barcelona/access/wheelchair_greenspace.geojson'
output_path = '../data/output/barcelona/isochrones_foot_greenspaces_wheelchair/'
output_filename = 'b_iso_foot_green_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)

with ramps

In [ ]:
input_file = '../data/output/barcelona/access/combined_greenspaces_wheelchair.geojson'
output_path = '../data/output/barcelona/isochrones_foot_greenspaces/'
output_filename = 'b_iso_foot_green'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)

### Munich

In [ ]:
input_file = '../data/output/munich/access/wheelchair_greenspace.geojson'
output_path = '../data/output/munich/isochrones_foot_greenspaces_wheelchair/'
output_filename = 'b_iso_foot_green_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)

In [ ]:
input_file = '../data/output/munich/access/combined_greenspaces_wheelchair.geojson'
output_path = '../data/output/munich/isochrones_foot_greenspaces/'
output_filename = 'b_iso_foot_green'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)

## Public spaces
### Barcelona

definitive wheelchair

In [ ]:
input_file = '../data/output/barcelona/access/wheelchair_publicspaces.geojson'
output_path = '../data/output/barcelona/isochrones_foot_public_spaces_wheelchair/'
output_filename = 'b_iso_foot_public_spaces_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)

with nulls

In [ ]:
input_file = '../data/output/barcelona/access/combined_public_spaces_wheelchair.geojson'
output_path = '../data/output/barcelona/isochrones_foot_public_spaces/'
output_filename = 'b_iso_foot_public_spaces'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)

### Munich

In [ ]:
input_file = '../data/output/munich/access/wheelchair_publicspaces.geojson'
output_path = '../data/output/munich/isochrones_foot_public_spaces_wheelchair/'
output_filename = 'b_iso_foot_public_spaces_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)


In [ ]:
input_file = '../data/output/munich/access/combined_public_spaces_wheelchair.geojson'
output_path = '../data/output/munich/isochrones_foot_public_spaces/'
output_filename = 'b_iso_foot_public_spaces'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)

## Public buildings
### Barcelona

definitive wheelchair

In [ ]:
input_file = '../data/output/barcelona/access/wheelchair_public_building.geojson'
output_path = '../data/output/barcelona/isochrones_foot_public_building_wheelchair/'
output_filename = 'b_iso_foot_public_building_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)

with nulls

In [ ]:
input_file = '../data/output/barcelona/access/combined_public_building_wheelchair.geojson'
output_path = '../data/output/barcelona/isochrones_foot_public_building_wheelchair/'
output_filename = 'b_iso_foot_public_building_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)

### Munich

In [ ]:
input_file = '../data/output/munich/access/wheelchair_public_building.geojson'
output_path = '../data/output/munich/isochrones_foot_public_building_wheelchair/'
output_filename = 'b_iso_foot_public_building_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)

In [ ]:
input_file = '../data/output/munich/access/combined_public_building_wheelchair.geojson'
output_path = '../data/output/munich/isochrones_foot_public_building_wheelchair/'
output_filename = 'b_iso_foot_public_building_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)